In [118]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [119]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [120]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'housing_unit_inventory_2022_20240718.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [121]:
parcels_davis = r'E:\Tasks\Housing_Unit_Inventory_Update\2022-Davis\Davis_LIR_Parcels_2022.gdb\Parcels_Davis_LIR'
parcels_weber = r'E:\Tasks\Housing_Unit_Inventory_Update\2022-Weber\Weber_Parcels_LIR_2022.gdb\Parcels_Weber_LIR_utm12'
parcels_sl = r'E:\Tasks\Housing_Unit_Inventory_Update\2022-Salt-Lake\Salt_Lake_LIR_Parcels_2022.gdb\Parcels_SaltLake_LIR_utm12'
housing_unit_inventory = r'E:\Tasks\Housing_Unit_Inventory_Update\housing_unit_inventory_2022_20240523.gdb\housing_unit_inventory_2022'

In [122]:
pts_weber = arcpy.management.FeatureToPoint(parcels_weber, os.path.join(gdb, 'parcels_weber_pts'), "INSIDE")
pts_davis = arcpy.management.FeatureToPoint(parcels_davis, os.path.join(gdb, 'parcels_davis_pts'), "INSIDE")
pts_sl = arcpy.management.FeatureToPoint(parcels_sl, os.path.join(gdb, 'parcels_sl_pts'), "INSIDE")

In [123]:
# get the unit id on each parcel point
target_features = pts_weber
join_features = housing_unit_inventory
output_features = os.path.join(gdb, "weber_sj")
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)
weber_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

target_features = pts_davis
join_features = housing_unit_inventory
output_features = os.path.join(gdb, "davis_sj")
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)
davis_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

target_features = pts_sl
join_features = housing_unit_inventory
output_features = os.path.join(gdb, "sl_sj")
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)
sl_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

In [124]:
df_weber = pd.DataFrame.spatial.from_featureclass(weber_sj[0])
df_davis = pd.DataFrame.spatial.from_featureclass(davis_sj[0])
df_sl = pd.DataFrame.spatial.from_featureclass(sl_sj[0])

In [125]:
df_weber = df_weber[['PARCEL_ID','TOTAL_MKT_VALUE','BLDG_SQFT','UNIT_ID','SHAPE']].copy()
df_davis = df_davis[['PARCEL_ID', 'TOTAL_MKT_VALUE','BLDG_SQFT','UNIT_ID','SHAPE']].copy()
df_sl = df_sl[['PARCEL_ID','TOTAL_MKT_VALUE','BLDG_SQFT','UNIT_ID','SHAPE']].copy()

In [126]:
df_weber = df_weber[df_weber['UNIT_ID'].isna() == False].copy()
df_davis = df_davis[df_davis['UNIT_ID'].isna() == False].copy()
df_sl = df_sl[df_sl['UNIT_ID'].isna() == False].copy()

In [127]:
dups_weber = df_weber[df_weber.duplicated(subset=['PARCEL_ID','UNIT_ID', 'TOTAL_MKT_VALUE'], keep=False)]
df_weber_dups_removed = df_weber.drop_duplicates(subset=['PARCEL_ID', 'TOTAL_MKT_VALUE'])

dups_davis = df_davis[df_davis.duplicated(subset=['PARCEL_ID','UNIT_ID', 'TOTAL_MKT_VALUE'], keep=False)]
df_davis_dups_removed = df_davis.drop_duplicates(subset=['PARCEL_ID', 'TOTAL_MKT_VALUE'])

dups_sl = df_sl[df_sl.duplicated(subset=['PARCEL_ID','UNIT_ID', 'TOTAL_MKT_VALUE'], keep=False)]
df_sl_dups_removed = df_sl.drop_duplicates(subset=['PARCEL_ID','UNIT_ID', 'TOTAL_MKT_VALUE'])

In [128]:
df_wfrc_dups_removed = pd.concat([df_weber_dups_removed, df_davis_dups_removed, df_sl_dups_removed])
dups_wfrc = pd.concat([dups_weber, dups_davis, dups_sl])

In [129]:
df_wfrc_dups_removed.spatial.to_featureclass(location=os.path.join(gdb, 'wfrc_parcel_data'),sanitize_columns=False)
dups_wfrc.spatial.to_featureclass(location=os.path.join(gdb, 'duplicate_values'),sanitize_columns=False)

'e:\\Projects\\Housing-Unit-Inventory-Explorer\\python\\Outputs\\scratch.gdb\\duplicate_values'

In [130]:
target_features = housing_unit_inventory
join_features = os.path.join(gdb, 'wfrc_parcel_data')
output_features = os.path.join(gdb, "wfrc_sj")
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# square ft
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

wfrc_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")
wfrc_sj_df = pd.DataFrame.spatial.from_featureclass(wfrc_sj[0])[['TOTAL_MKT_VALUE','BLDG_SQFT','UNIT_ID']].copy()

In [148]:
hui_df = pd.DataFrame.spatial.from_featureclass(housing_unit_inventory)
hui_df = hui_df.merge(wfrc_sj_df, on='UNIT_ID', how='left')
# hui_df.loc[hui_df['TOTAL_MKT_VALUE'] > 0, 'TMV_DIFF'] = hui_df['TOTAL_MKT_VALUE'] - hui_df['TOT_VALUE']
# hui_df.loc[hui_df['BLDG_SQFT'] > 0, 'SQFT_DIFF'] = hui_df['BLDG_SQFT'] - hui_df['TOT_BD_FT2']
hui_df.loc[hui_df['TOTAL_MKT_VALUE'] > 0, 'TOT_VALUE'] = round(hui_df['TOTAL_MKT_VALUE'])
hui_df.loc[hui_df['BLDG_SQFT'] > 0, 'TOT_BD_FT2'] = round(hui_df['BLDG_SQFT'])
hui_df.drop(['TOTAL_MKT_VALUE','BLDG_SQFT'], axis=1, inplace=True)


In [151]:
hui_df.spatial.to_featureclass(location=os.path.join(gdb2, 'housing_unit_inventory_2022'), sanitize_columns=False)  

Could not insert the row because of error message: value #9 - unsupported type: NAType. Recheck your data.


'e:\\Projects\\Housing-Unit-Inventory-Explorer\\python\\Outputs\\housing_unit_inventory_2022_20240718.gdb\\housing_unit_inventory_2022'